In [ ]:
# =========================================================
# AUTOENCODER DIDÁTICO - MNIST
# =========================================================

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, datasets

In [ ]:
# ---------------------------------------------------------
# 1. Carregar e preparar dados
# ---------------------------------------------------------
(x_train, _), (x_test, _) = datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

# Flatten das imagens 28x28 → vetor 784
x_train = x_train.reshape((len(x_train), 784))
x_test  = x_test.reshape((len(x_test), 784))

In [ ]:
# ---------------------------------------------------------
# 2. Definir o modelo do Autoencoder
# ---------------------------------------------------------
encoding_dim = 32  # Dimensão comprimida (redução de 784 → 32)

input_img = layers.Input(shape=(784,))
encoded = layers.Dense(encoding_dim, activation='relu')(input_img)
decoded = layers.Dense(784, activation='sigmoid')(encoded)

autoencoder = models.Model(input_img, decoded)

# Também criamos o codificador separadamente (para visualização posterior)
encoder = models.Model(input_img, encoded)

In [ ]:
# ---------------------------------------------------------
# 3. Compilar e treinar
# ---------------------------------------------------------
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

history = autoencoder.fit(
    x_train, x_train,
    epochs=20,
    batch_size=256,
    shuffle=True,
    validation_data=(x_test, x_test)
)

In [ ]:
# ---------------------------------------------------------
# 4. Visualizar resultados
# ---------------------------------------------------------
# Codificar e reconstruir
encoded_imgs = encoder.predict(x_test)
decoded_imgs = autoencoder.predict(x_test)

# Mostrar imagens originais e reconstruídas
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # Original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    plt.title("Original")
    plt.axis("off")

    # Reconstruída
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28), cmap='gray')
    plt.title("Reconstruída")
    plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
# =========================================================
# VISUALIZAÇÃO DO GARGALO (ESPAÇO LATENTE)
# =========================================================
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from tensorflow.keras import datasets

# Se o encoder e x_test já existirem no ambiente, pule o carregamento.
# Caso contrário, descomente as 3 linhas abaixo:
# (x_train, _), (x_test, y_test) = datasets.mnist.load_data()
# x_test = x_test.astype("float32") / 255.0
# x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

# ---------------------------------------------------------
# 1️⃣ Gerar embeddings (gargalo)
# ---------------------------------------------------------
encoded_imgs = encoder.predict(x_test)

# ---------------------------------------------------------
# 2️⃣ Reduzir para 2D (PCA ou t-SNE)
# ---------------------------------------------------------
method = "TSNE"  # ou "PCA"

if method == "PCA":
    reducer = PCA(n_components=2)
else:
    reducer = TSNE(n_components=2, random_state=42, perplexity=30)

latent_2d = reducer.fit_transform(encoded_imgs)

# ---------------------------------------------------------
# 3️⃣ Visualizar o espaço latente colorido por rótulo
# ---------------------------------------------------------
plt.figure(figsize=(8, 6))
plt.scatter(latent_2d[:, 0], latent_2d[:, 1],
            s=10, c=y_test, cmap='tab10', alpha=0.8)
plt.colorbar(label="Dígito real")
plt.title(f"Espaço Latente do Autoencoder ({method})")
plt.xlabel("Dimensão 1")
plt.ylabel("Dimensão 2")
plt.grid(True)
plt.show()
